In [2]:
!pip install pandas rasterio netCDF4 rioxarray affine
from google.colab import drive
import geopandas as gpd
drive.mount('/content/drive')
%cd /content/drive/My\ Drive/LST\ data/sentinel

import os
import re
import sys
import random
from pathlib import Path

import requests
import json
import xml.etree.ElementTree as ET
import certifi

import pandas as pd
import numpy as np

import rasterio
import matplotlib.pyplot as plt
import matplotlib.image
from rasterio.windows import Window
from affine import Affine

import netCDF4 as nc

import rioxarray
from rasterio.control import GroundControlPoint

dates = pd.read_csv("Copy of Heat Wave Days(90 3D).csv",usecols=["Date"])["Date"].to_list()

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).
/content/drive/My Drive/LST data/sentinel


In [3]:
url_dataspace = "https://catalogue.dataspace.copernicus.eu/odata/v1"

# Filtering
satellite = "SENTINEL-3"
level= "LST"
cloud_cover_max = 0.2

aoi_point ="POINT(-120.9970 37.6393)"
aoi_polygon = "POLYGON ((-74.255591 40.496134,-74.255591 40.915533,-73.700009 40.915533,-73.700009 40.496134,-74.255591 40.496134))"

product_id_df = pd.DataFrame()
for start_date in dates:
  end_date = start_date
  start_date_full =start_date+"T00:00:00.000Z"
  end_date_full = end_date +"T23:59:59.000Z"

  query = f"{url_dataspace}/Products?$filter=Collection/Name eq '{satellite}' \
        and Attributes/OData.CSC.StringAttribute/any(att:att/Name eq 'productType' \
        and att/OData.CSC.StringAttribute/Value eq '{level}') \
        and OData.CSC.Intersects(area=geography'SRID=4326;{aoi_polygon}') \
        and ContentDate/Start gt {start_date_full} \
        and ContentDate/Start lt {end_date_full}"
  response = requests.get(query).json()
  result = pd.DataFrame.from_dict(response["value"])
  if result.empty:
    continue
  filtered_df = result[result["Name"].str.split("_").str[8].str.contains("T12|T13|T14|T15|T16|T17")]
  filtered_df = filtered_df.reset_index(drop=True)

  product_id_df = pd.concat([product_id_df, filtered_df], ignore_index=True)
auth_server_url = "https://identity.dataspace.copernicus.eu/auth/realms/CDSE/protocol/openid-connect/token"
data = {
    "client_id": "cdse-public",
    "grant_type": "password",
    "username": "bigseaboard@gmail.com",
    "password": "Hql20020106!",
}


products_dict = [
    {
        "product_id": row["Id"],
        "product_name": row["Name"]
    }
    for _, row in product_id_df.iterrows()
]

In [4]:
products_dict

[{'product_id': '74ff80a2-1879-530f-854b-3314e5d4f695',
  'product_name': 'S3A_SL_2_LST____20170517T142357_20170517T160456_20180929T235123_6059_017_367______LR1_R_NT_003.SEN3'},
 {'product_id': '452aa845-2be2-5236-bca6-0e839325a014',
  'product_name': 'S3A_SL_2_LST____20170519T151234_20170519T165333_20180929T235432_6059_018_011______LR1_R_NT_003.SEN3'},
 {'product_id': '76fcd6e8-54f0-5e09-b1b6-8099f5977734',
  'product_name': 'S3A_SL_2_LST____20170611T151619_20170611T165718_20180930T003839_6059_018_339______LR1_R_NT_003.SEN3'},
 {'product_id': '3e621ef1-84b0-506c-8475-1d62e615ed0d',
  'product_name': 'S3A_SL_2_LST____20170612T145008_20170612T163107_20180930T004020_6059_018_353______LR1_R_NT_003.SEN3'},
 {'product_id': '2b04d18f-e8d9-599d-a069-c9b7d6ae3045',
  'product_name': 'S3A_SL_2_LST____20170613T142357_20170613T160456_20180930T004258_6059_018_367______LR1_R_NT_003.SEN3'},
 {'product_id': '67c28052-59fa-59f5-9b76-71bad9e588d9',
  'product_name': 'S3A_SL_2_LST____20170719T153116_201

In [ ]:
import time
folder_path = "LZT zips"
os.makedirs(folder_path, exist_ok=True)

for product in products_dict:

  product_id = product["product_id"]
  product_name = product["product_name"]
  file_path = os.path.join(folder_path, f"{product_name}.zip")
  # 如果文件已经存在，就跳过下载
  if os.path.exists(file_path):
      print(f"文件 {file_path} 已存在，跳过下载。")
      continue

  response = requests.post(auth_server_url, data=data, verify=True, allow_redirects=False)
  access_token = json.loads(response.text)["access_token"]

  url = f"https://zipper.dataspace.copernicus.eu/odata/v1/Products({product_id})/$value"
  headers = {"Authorization": f"Bearer {access_token}"}

  # 使用 Session 来发送请求
  session = requests.Session()
  session.headers.update(headers)

  try:
      # 发送请求
      response = session.get(url, stream=True, allow_redirects=False)

      # 如果响应状态码不是 200，暂停 2 分钟再试
      if response.status_code != 200:
          print(f"下载失败，状态码：{response.status_code}，等待 2 分钟后重试。")
          time.sleep(120)  # 等待两分钟再试
          continue  # 重新进行下载操作

      # 否则，正常保存文件
      with open(file_path, "wb") as file:
          for chunk in response.iter_content(chunk_size=8192):
              if chunk:
                  file.write(chunk)

  except requests.exceptions.RequestException as e:
      print(f"请求错误：{e}. 等待 2 分钟后重试。")
      time.sleep(80)  # 如果请求异常，也暂停 2 分钟

文件 LZT zips/S3A_SL_2_LST____20170517T142357_20170517T160456_20180929T235123_6059_017_367______LR1_R_NT_003.SEN3.zip 已存在，跳过下载。
文件 LZT zips/S3A_SL_2_LST____20170519T151234_20170519T165333_20180929T235432_6059_018_011______LR1_R_NT_003.SEN3.zip 已存在，跳过下载。
文件 LZT zips/S3A_SL_2_LST____20170611T151619_20170611T165718_20180930T003839_6059_018_339______LR1_R_NT_003.SEN3.zip 已存在，跳过下载。
文件 LZT zips/S3A_SL_2_LST____20170612T145008_20170612T163107_20180930T004020_6059_018_353______LR1_R_NT_003.SEN3.zip 已存在，跳过下载。
文件 LZT zips/S3A_SL_2_LST____20170613T142357_20170613T160456_20180930T004258_6059_018_367______LR1_R_NT_003.SEN3.zip 已存在，跳过下载。
文件 LZT zips/S3A_SL_2_LST____20170719T153116_20170719T171215_20180930T015116_6059_020_111______LR1_R_NT_003.SEN3.zip 已存在，跳过下载。
文件 LZT zips/S3A_SL_2_LST____20170720T150505_20170720T164605_20180930T015358_6059_020_125______LR1_R_NT_003.SEN3.zip 已存在，跳过下载。
文件 LZT zips/S3A_SL_2_LST____20170721T143854_20170721T161954_20180930T015618_6059_020_139______LR1_R_NT_003.SEN3.zip 已存

In [ ]:
response

<Response [401]>